<a href="https://colab.research.google.com/github/pwilczewski/twostep/blob/main/trouble_with_next_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers accelerate bitsandbytes

from transformers import AutoModelForCausalLM, PreTrainedTokenizerFast, BitsAndBytesConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.8 MB/s eta 0:00:00


Using Llama 3.1 8B model to demonstrate issues with next token prediction

In [2]:
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

from google.colab import userdata
model_name = "meta-llama/Llama-3.1-8B"
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B", device_map="auto", token=userdata.get('HF_TOKEN'), quantization_config=bnb_config)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, token=userdata.get('HF_TOKEN'))
tokenizer.pad_token = tokenizer.eos_token
model.generation_config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Function to generate predictions for a list of contexts

In [49]:
def generate_text(model, tokenizer, contexts, temperature=1.0, num_beams=1, num_tokens=10):
    predictions = []

    model.generation_config.temperature = temperature
    model.generation_config.num_beams = num_beams
    model.generation_config.num_return_sequences = 1

    for text in contexts:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=num_tokens)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(generated_text)

    return predictions


**Test #1:** predictions with very low temperature and no beam search

In [54]:
contexts = ["The capital of France is",
             "The capital of Germany is",
             "The capital of Spain is",
             "The capital of Ireland is"]

predictions = generate_text(model, tokenizer, contexts, temperature=0.0001)

print('\n'.join(predictions))

The capital of France is Paris. The capital of France is Paris. The
The capital of Germany is Berlin. It is the largest city in the country
The capital of Spain is Madrid, and the capital of the Basque Country
The capital of Ireland is Dublin, and the country is divided into four provinces


**Test #2:** predictions with low temperature and 8 beams

In [53]:
predictions = generate_text(model, tokenizer, contexts, temperature=0.1, num_beams=8)

print('\n'.join(predictions))

The capital of France is Paris, which is located in the north of the
The capital of Germany is Berlin. It is the largest city in the country
The capital of Spain is Madrid. It is located in the center of the
The capital of Ireland is Dublin, which is located on the east coast of
